In [1]:
import pandas as pd
import wrds

In [2]:
db =wrds.Connection()

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [3]:
sp500 = db.raw_sql("""
                        select a.*, b.date, b.ret
                        from crsp.dsp500list as a,
                        crsp.dsf as b
                        where a.permno=b.permno
                        and b.date >= a.start and b.date<= a.ending
                        and b.date>='01/01/2000'
                        order by date;
                        """, date_cols=['start', 'ending', 'date'])

In [4]:
sp500.head(5)

,permno,start,ending,date,ret
0,64995,1994-03-02,2023-12-29,2000-01-03,-0.062147
1,57665,1988-11-03,2023-12-29,2000-01-03,-0.029004
2,85908,1998-04-01,2000-12-08,2000-01-03,-0.061947
3,60628,1980-11-06,2023-12-29,2000-01-03,0.042748
4,19350,1957-03-01,2023-12-29,2000-01-03,-0.011527


In [5]:
mse = db.raw_sql("""
                        select comnam, ncusip, namedt, nameendt, 
                        permno, shrcd, exchcd, hsiccd, ticker
                        from crsp.msenames
                        """, date_cols=['namedt', 'nameendt'])

# if nameendt is missing then set to today date
mse['nameendt']=mse['nameendt'].fillna(pd.to_datetime('today'))

In [6]:
mse.sample(5)

,comnam,ncusip,namedt,nameendt,permno,shrcd,exchcd,hsiccd,ticker
22800,ISHARES TRUST,46435U15,2020-09-04,2023-12-15,18701,73,5,6726,IBHC
72057,COPERNIC INC,21727W20,2009-09-14,2009-10-11,78690,12,3,5065,CNIC
89490,LETS TALK CELLULAR AND WIRLS INC,52726010,1999-11-19,2000-01-19,85563,11,3,5990,LTCW
60193,STANSBURY MINING CORP,85472610,1981-07-15,1982-10-31,72574,11,3,1490,STBY
104286,ECOST COM INC,27922W10,2004-08-27,2004-09-09,90315,11,3,5961,ECST


In [7]:
# Merge with SP500 data
sp500_full = pd.merge(sp500, mse, how = 'left', on = 'permno')

# Impose the date range restrictions
sp500_full = sp500_full.loc[(sp500_full.date>=sp500_full.namedt) \
                            & (sp500_full.date<=sp500_full.nameendt)]
sp500_full.sample(5)

,permno,start,ending,date,ret,comnam,ncusip,namedt,nameendt,shrcd,exchcd,hsiccd,ticker
21567047,17005,1957-03-01,2023-12-29,2022-10-13,0.025729,C V S HEALTH CORP,12665010,2014-09-04,2023-03-23,11,1,5912,CVS
14308762,68857,2010-12-20,2016-06-23,2014-12-11,-0.018609,CABLEVISION SYSTEMS CORP,12686C10,2005-11-08,2016-06-21,11,1,4841,CVC
15368906,75489,1998-10-07,2017-09-15,2016-01-29,0.021764,STAPLES INC,85503010,2004-06-10,2017-09-12,11,3,5940,SPLS
13295375,42534,1986-05-08,2016-09-02,2013-11-13,0.011042,JOHNSON CONTROLS INC,47836610,2013-03-20,2014-01-22,11,1,2531,JCI
5634631,92655,1994-07-01,2023-12-29,2005-12-01,0.026896,UNITEDHEALTH GROUP INC,91324P10,2004-06-10,2023-12-29,11,1,6324,UNH


In [8]:
# Linking with Compustat through CCM

ccm=db.raw_sql("""
                  select gvkey, liid as iid, lpermno as permno, linktype, linkprim, 
                  linkdt, linkenddt
                  from crsp.ccmxpf_linktable
                  where substr(linktype,1,1)='L'
                  and (linkprim ='C' or linkprim='P')
                  """, date_cols=['linkdt', 'linkenddt'])

# if linkenddt is missing then set to today date
ccm['linkenddt']=ccm['linkenddt'].fillna(pd.to_datetime('today'))

In [9]:
# Merge the CCM data with S&P500 data
# First just link by matching PERMNO
sp500ccm = pd.merge(sp500_full, ccm, how='left', on=['permno'])

# Then set link date bounds
sp500ccm = sp500ccm.loc[(sp500ccm['date']>=sp500ccm['linkdt'])\
                        &(sp500ccm['date']<=sp500ccm['linkenddt'])]
sp500ccm.sample(5)

,permno,start,ending,date,ret,comnam,ncusip,namedt,nameendt,shrcd,exchcd,hsiccd,ticker,gvkey,iid,linktype,linkprim,linkdt,linkenddt
908449,48506,1979-12-27,2023-12-29,2005-05-27,0.011330,MOODYS CORP,61536910,2004-06-10,2006-06-29,11,1,7323,MCO,139665,01,LC,P,2000-10-03,2024-04-02 15:17:35.196306
1256951,75034,1987-04-09,2023-12-29,2007-07-23,0.017597,BAKER HUGHES INC,05722410,2005-06-16,2014-01-06,11,1,3533,BHI,001976,01,LC,P,1987-04-24,2017-07-04 00:00:00.000000
722980,10942,1999-10-27,2007-01-31,2004-04-15,-0.018085,COMVERSE TECHNOLOGY INC,20586240,2000-08-01,2004-06-09,11,3,3661,CMVT,012884,01,LC,P,1986-12-15,2007-01-31 00:00:00.000000
993785,77768,1992-07-06,2018-10-30,2005-12-02,-0.002066,PRAXAIR INC,74005P10,2004-06-10,2018-10-30,11,1,2813,PX,025124,01,LC,P,1992-07-06,2018-10-30 00:00:00.000000
3439865,22592,1957-03-01,2023-12-29,2021-06-25,0.007710,3M CO,88579Y10,2020-10-23,2022-07-04,11,1,3841,MMM,007435,01,LC,P,1962-01-31,2024-04-02 15:17:35.196306


In [10]:
# Rearrange columns for final output

sp500ccm = sp500ccm.drop(columns=['namedt', 'nameendt', \
                                  'linktype', 'linkprim', 'linkdt', 'linkenddt'])
sp500ccm = sp500ccm[['date', 'permno', 'comnam', 'ncusip', 'shrcd', 'exchcd', 'hsiccd', 'ticker', \
                     'gvkey', 'iid', 'start', 'ending', 'ret']]
sp500ccm.sample(5)

,date,permno,comnam,ncusip,shrcd,exchcd,hsiccd,ticker,gvkey,iid,start,ending,ret
928814,2005-07-14,14277,SCHLUMBERGER LTD,80685710,12,1,5651,SLB,009465,01,1965-03-04,2023-12-29,-0.029139
2246363,2013-10-15,48486,LAM RESH CORP,51280710,11,3,3559,LRCX,006565,01,2012-06-05,2023-12-29,-0.012758
686768,2004-01-27,57904,AFLAC INC,00105510,11,1,6321,AFL,001449,01,1999-05-28,2023-12-29,0.000000
1810694,2011-01-05,86868,GOLDMAN SACHS GROUP INC,38141G10,11,1,6211,GS,114628,01,2002-07-22,2023-12-29,0.005315
805546,2004-10-13,83332,LUCENT TECHNOLOGIES INC,54946310,11,1,3661,LU,062599,01,1996-10-01,2006-11-30,0.011834


In [11]:
cnt = sp500ccm.groupby(['date'])['permno'].count().reset_index().rename(columns={'permno':'npermno'})
cnt.sample(4)

,date,npermno
5740,2022-10-25,500
4036,2016-01-20,501
2238,2008-11-25,501
398,2001-08-01,500


In [13]:
sp500ccm.sample(50)

,date,permno,comnam,ncusip,shrcd,exchcd,hsiccd,ticker,gvkey,iid,start,ending,ret
1542910,2009-04-30,61815,NOBLE ENERGY INC,65504410,11,1,1311,NBL,007912,01,2007-10-08,2020-10-09,-0.058404
3630572,2022-09-21,45356,JOHNSON CONTROLS INTL PLC,G5150210,12,1,5065,JCI,006268,01,2010-08-27,2023-12-29,-0.010171
525740,2003-02-05,75646,ALLERGAN INC,01849010,11,1,2834,AGN,015708,01,1992-02-10,2015-03-20,-0.005677
374147,2002-03-11,76076,CISCO SYSTEMS INC,17275R10,11,3,3674,CSCO,020779,01,1993-12-31,2023-12-29,-0.021348
3195610,2019-11-25,86996,S B A COMMUNICATIONS CORP NEW,78410G10,18,3,4899,SBAC,121382,01,2017-09-01,2023-12-29,-0.003486
2143731,2013-02-19,68591,ST JUDE MEDICAL INC,79084910,11,1,3845,STJ,009372,01,1989-12-01,2017-01-04,0.002890
3139351,2019-07-17,22293,CORNING INC,21935010,11,1,3229,GLW,003532,01,1957-03-01,2023-12-29,-0.010511
401104,2002-05-08,24766,NORTHROP GRUMMAN CORP,66680710,11,1,3812,NOC,007985,01,1985-06-13,2023-12-29,0.017912
967719,2005-10-06,51043,FEDERAL NATIONAL MORTGAGE ASSN,31358610,11,1,6111,FNM,004601,01,1988-08-11,2008-09-10,0.008169
1527941,2009-03-27,77668,EXPRESS SCRIPTS INC,30218210,11,3,8093,ESRX,025356,01,2003-09-26,2018-12-21,-0.050206
